In [1]:
# ==========================================================
# Artwork Analyzer v16 – Clear Overlays + Fixed Black Panel
# ==========================================================
# Local-only (OpenCV + NumPy + Pillow)
# Output = single PNG:
#   • LEFT: artwork with ONLY colored overlays (no text)
#   • RIGHT: fixed-width black analysis panel
#            - legend
#            - per-indicator captions (+ “weak/omitted” notes)
#            - paragraph-style curatorial summary
# Also saves: curatorial_summary.txt, metrics.json
# ==========================================================

import os, json, time, textwrap, math
import numpy as np
import cv2
from PIL import Image
from math import atan2, degrees

PANEL_W = 400  # fixed width for readability

# -----------------------------
# Utilities
# -----------------------------
def load_rgb(path):
    img = Image.open(path).convert("RGB")
    return np.array(img)

def ensure_dir(p): os.makedirs(p, exist_ok=True)

def to_uint8(img_f32):
    return np.clip(img_f32*255.0, 0, 255).astype(np.uint8)

def safe_gamma(img_u8):
    # perceptual tweak without blowing highlights
    img = img_u8.astype(np.float32)/255.0
    m = float(img.mean())
    if m < 0.22:  # dark
        img = np.power(img, 0.65)
    elif m > 0.85:  # bright
        img = np.power(img, 1.15)
    return to_uint8(np.clip(img, 0, 1))

def resize_min_pixels(img, target_pixels=1_600_000):
    H, W = img.shape[:2]
    cur = H * W
    if cur >= target_pixels:
        return img, 1.0
    scale = math.sqrt(target_pixels / float(cur))
    newW, newH = int(W*scale), int(H*scale)
    new = cv2.resize(img, (newW, newH), interpolation=cv2.INTER_CUBIC)
    return new, scale

def wrap_text_lines(text, width_chars=80):
    return textwrap.wrap(text, width=width_chars)

def json_safe(obj):
    if isinstance(obj, dict):
        return {k: json_safe(v) for k, v in obj.items()}
    elif isinstance(obj, (list, tuple)):
        return [json_safe(x) for x in obj]
    elif isinstance(obj, (np.generic, np.number)):
        return obj.item()
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# Normalize x to 0..1 based on [lo, hi]
def norm01(x, lo, hi):
    if hi <= lo: return 0.0
    v = (x - lo) / (hi - lo)
    return float(np.clip(v, 0.0, 1.0))
# -----------------------------
# CIEDE2000 (for center distances)
# -----------------------------
def ciede2000(Lab1, Lab2):
    L1,a1,b1 = Lab1; L2,a2,b2 = Lab2
    C1 = np.sqrt(a1*a1 + b1*b1); C2 = np.sqrt(a2*a2 + b2*b2)
    avg_C = (C1 + C2)/2.0
    G = 0.5*(1 - np.sqrt((avg_C**7)/((avg_C**7)+25**7)))
    a1p, a2p = (1+G)*a1, (1+G)*a2
    C1p, C2p = np.sqrt(a1p*a1p + b1*b1), np.sqrt(a2p*a2p + b2*b2)
    h1p = (np.degrees(np.arctan2(b1, a1p)) + 360) % 360
    h2p = (np.degrees(np.arctan2(b2, a2p)) + 360) % 360
    dLp = L2-L1; dCp = C2p-C1p
    dhp = h2p-h1p
    if C1p*C2p == 0: dhp = 0
    else:
        if dhp > 180: dhp -= 360
        elif dhp < -180: dhp += 360
    dHp = 2*np.sqrt(C1p*C2p)*np.sin(np.radians(dhp)/2.0)
    avg_Lp = (L1+L2)/2.0
    if C1p*C2p == 0:
        avg_hp = h1p + h2p
    else:
        if abs(h1p-h2p) > 180:
            avg_hp = (h1p+h2p+360)/2.0 if (h1p+h2p)<360 else (h1p+h2p-360)/2.0
        else:
            avg_hp = (h1p+h2p)/2.0
    T = (1 - 0.17*np.cos(np.radians(avg_hp-30)) +
         0.24*np.cos(np.radians(2*avg_hp)) +
         0.32*np.cos(np.radians(3*avg_hp+6)) -
         0.20*np.cos(np.radians(4*avg_hp-63)))
    d_ro = 30*np.exp(-((avg_hp-275)/25)**2)
    avg_Cp = (C1p + C2p)/2.0
    Rc = 2*np.sqrt(( (avg_Cp**7) / ((avg_Cp**7)+25**7) ))
    Sl = 1 + 0.015*(avg_Lp-50)**2 / np.sqrt(20 + (avg_Lp-50)**2)
    Sc = 1 + 0.045*avg_Cp
    Sh = 1 + 0.015*avg_Cp*T
    Rt = -np.sin(np.radians(2*d_ro))*Rc
    return float(np.sqrt((dLp/Sl)**2 + (dCp/Sc)**2 + (dHp/Sh)**2 + Rt*(dCp/Sc)*(dHp/Sh)))

def _kmeans_lab(lab_img, k, attempts=10):
    H, W = lab_img.shape[:2]
    Z = lab_img.reshape(-1,3).astype(np.float32)
    crit = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 80, 0.2)
    _r, labels, centers = cv2.kmeans(Z, k, None, crit, attempts, cv2.KMEANS_PP_CENTERS)
    return labels.reshape(H, W), centers.astype(np.float32)

def _palette_quality(centers, entropy):
    if centers.shape[0] < 2: return entropy, 0.0
    dEs = [ciede2000(centers[i], centers[j]) for i in range(len(centers)) for j in range(i+1, len(centers))]
    mean_dE = float(np.mean(dEs)) if dEs else 0.0
    spread = float(np.std(centers, axis=0).mean())
    return 0.6*(0.7*mean_dE + 0.3*spread) + 0.4*entropy, mean_dE

def color_analysis(img, ks=(6,8,10)):
    H, W = img.shape[:2]
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hue = (hsv[:,:,0].astype(np.float32)*2.0) % 360.0
    hue_hist, _ = np.histogram(hue, bins=36, range=(0,360))
    hue_hist = hue_hist.astype(np.float32)

    best = None; best_score = -1
    for k in ks:
        labels2, centers = _kmeans_lab(lab, k)
        counts = np.bincount(labels2.reshape(-1), minlength=k).astype(np.float32)
        frac = counts/(H*W + 1e-6)
        p = frac[frac > 0]
        entropy = -float(np.sum(p*np.log2(p)))
        score, mean_dE = _palette_quality(centers, entropy)
        if score > best_score:
            best_score = score
            best = (k, labels2, centers, counts, entropy, mean_dE)

    k_sel, labels2, centers, counts, entropy, mean_dE = best
    masks = [(labels2 == i).astype(np.uint8)*255 for i in range(k_sel)]
    rgb_centers = cv2.cvtColor(centers.reshape(-1,1,3).astype(np.uint8), cv2.COLOR_Lab2RGB).reshape(-1,3)
    palette = [tuple(map(int, c)) for c in rgb_centers]

    a, b = centers[:,1]-128, centers[:,2]-128
    hues_c = (np.degrees(np.arctan2(b, a)) + 360) % 360
    warm_ratio = float(((hues_c < 90) | (hues_c > 330)).sum()/len(hues_c))
    cool_ratio = float(((hues_c > 180) & (hues_c < 300)).sum()/len(hues_c))
    harmony = "mixed"
    if np.ptp(hues_c) < 35: harmony = "analogous"
    elif any(abs(hues_c[i]-hues_c[j])%360 > 150 for i in range(len(hues_c)) for j in range(i+1, len(hues_c))):
        harmony = "complementary"

    # ΔE proxy via Lab gradient magnitude
    labf = lab.astype(np.float32)
    gxL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 1, 0); gyL = cv2.Sobel(labf[:,:,0], cv2.CV_32F, 0, 1)
    gxa = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 1, 0); gya = cv2.Sobel(labf[:,:,1], cv2.CV_32F, 0, 1)
    gxb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 1, 0); gyb = cv2.Sobel(labf[:,:,2], cv2.CV_32F, 0, 1)
    grad_mag = np.sqrt((gxL**2 + gyL**2) + (gxa**2 + gya**2) + (gxb**2 + gyb**2))
    contrast_map = (grad_mag - grad_mag.min())/(grad_mag.max()-grad_mag.min()+1e-6)

    return dict(
        palette_rgb=palette,
        masks=masks,
        spatial_cohesion=float((counts/(H*W+1e-6)).max()),
        palette_entropy=float(entropy),
        mean_deltaE=float(mean_dE),
        max_deltaE=float(np.max([ciede2000(centers[i], centers[j]) for i in range(len(centers)) for j in range(i+1,len(centers))])) if len(centers)>1 else 0.0,
        warm_ratio=warm_ratio, cool_ratio=cool_ratio, harmony=harmony,
        contrast_map=contrast_map,
        hue_hist=hue_hist,
        k_selected=int(k_sel), ks=list(ks)
    )
# -----------------------------
# Texture: Gabor pyramid (multi-scale, multi-angle)
# -----------------------------
def gabor_pyramid(gray, scales=(3,5,7,9,11,13), orientations=12):
    H, W = gray.shape
    angles = np.linspace(0, 180, orientations, endpoint=False)
    combined = np.zeros((H,W), np.float32)
    angle_energy = np.zeros(orientations, np.float32)

    for oi, theta in enumerate(angles):
        theta_rad = np.deg2rad(theta)
        e_sum = np.zeros((H,W), np.float32)
        for k in scales:
            kernel = cv2.getGaborKernel((21,21), sigma=k, theta=theta_rad, lambd=10, gamma=0.5, psi=0)
            resp = cv2.filter2D(gray, cv2.CV_32F, kernel)
            e_sum += resp*resp
        combined = np.maximum(combined, e_sum)
        angle_energy[oi] = e_sum.mean()

    anisotropy = float((angle_energy.max() - angle_energy.min()) /(angle_energy.mean()+1e-6))
    dom_angle = float(angles[int(np.argmax(angle_energy))])
    gmap = (combined - combined.min())/(combined.max()-combined.min()+1e-6)

    # simple entropy & roughness
    hist, _ = np.histogram(gray, bins=256, range=(0,256))
    hist = hist.astype(np.float32); hist /= (hist.sum()+1e-6)
    entropy = -float(np.sum(hist*np.log2(hist + 1e-12)))
    roughness = float(gray.std())

    return dict(
        gabor_angles=angles, gabor_energy=angle_energy, gabor_map=gmap,
        gabor_anisotropy=anisotropy, gabor_dom_angle=dom_angle,
        lbp_entropy=entropy, lbp_roughness=roughness
    )

# -----------------------------
# Signal strength helpers for visibility
# -----------------------------
def texture_strength(T):
    # combine anisotropy + roughness
    a = norm01(T['gabor_anisotropy'], 0.15, 0.85)
    r = norm01(T['lbp_roughness'], 8.0, 35.0)
    return 0.6*a + 0.4*r

def contrast_strength(C):
    return norm01(C['mean_deltaE'], 6.0, 28.0)

def saliency_strength(mask):
    # normalized area of subject mask
    area = float(mask.sum()) / 255.0
    return norm01(area, 0.04 * mask.size, 0.45 * mask.size)
# -----------------------------
# Saliency (two-scale spectral residual) + fused subject
# -----------------------------
def spectral_saliency(grayf, blur=(9,9), sigma=2):
    fft = np.fft.fft2(grayf)
    loga = np.log(np.abs(fft)+1e-8)
    spec = np.exp((loga - cv2.blur(loga,(3,3))) + 1j*np.angle(fft))
    sal = np.abs(np.fft.ifft2(spec))**2
    sal = cv2.GaussianBlur(sal, blur, sigma)
    sal = (sal - sal.min())/(sal.max()-sal.min()+1e-6)
    return (sal*255).astype(np.uint8)

def subject_from_saliency_fused(s1, s2):
    H, W = s1.shape
    m1 = cv2.threshold(s1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    m2 = cv2.threshold(s2, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    m = cv2.bitwise_and(m1, m2)
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((5,5), np.uint8))
    m = cv2.morphologyEx(m, cv2.MORPH_CLOSE, np.ones((7,7), np.uint8))
    cnts,_ = cv2.findContours(m, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts:
        x,y,w,h = cv2.boundingRect(max(cnts, key=cv2.contourArea))
    else:
        x,y,w,h = W//4, H//4, W//2, H//2
        cv2.rectangle(m, (x,y), (x+w,y+h), 255, -1)
    return dict(mask=m, bbox=(x,y,w,h))

def saliency_analysis(img):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY).astype(np.float32)
    s_coarse = spectral_saliency(cv2.GaussianBlur(g,(0,0), 3.0), blur=(11,11), sigma=3)
    s_fine   = spectral_saliency(cv2.GaussianBlur(g,(0,0), 1.0), blur=(7,7),  sigma=1.5)
    fused = np.maximum(s_coarse, s_fine)
    sub = subject_from_saliency_fused(s_coarse, s_fine)
    Y, X = np.indices(fused.shape); w = fused.astype(np.float32)+1
    cx, cy = int((X*w).sum()/w.sum()), int((Y*w).sum()/w.sum())
    return dict(saliency_map=fused, focal=(cx,cy), subject_mask=sub["mask"], subject_bbox=sub["bbox"])
# -----------------------------
# Composition (multisweep Hough + vanishing clustering)
# -----------------------------
def _line_intersection(p1, p2):
    (x1,y1,x2,y2), (x3,y3,x4,y4) = p1, p2
    den = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
    if abs(den) < 1e-6: return None
    px = ((x1*y2 - y1*x2)*(x3-x4) - (x1-x2)*(x3*y4 - y3*x4))/den
    py = ((x1*y2 - y1*x2)*(y3-y4) - (y1-y2)*(x3*y4 - y3*x4))/den
    return (px, py)

def _cluster_points(points, grid):
    if not points: return None
    pts = np.array(points, dtype=np.float32)
    xs, ys = pts[:,0], pts[:,1]
    xbin = np.round(xs/grid).astype(int)
    ybin = np.round(ys/grid).astype(int)
    keys = xbin*100000 + ybin
    uniq, counts = np.unique(keys, return_counts=True)
    best_key = uniq[np.argmax(counts)]
    mask = (keys == best_key)
    bx = float(xs[mask].mean()); by = float(ys[mask].mean())
    return (bx, by)

def composition_analysis(img, focal_xy):
    g = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    H, W = g.shape
    edges = cv2.Canny(g, 80, 200)

    # multiple sweeps
    lines = []
    params = [
        (1, np.pi/180, 100, min(H,W)//12, 15),
        (1, np.pi/180,  80, min(H,W)//14, 20),
        (1, np.pi/180, 120, min(H,W)//10, 15)
    ]
    for rho, theta, thr, minL, gap in params:
        linesP = cv2.HoughLinesP(edges, rho, theta, thr, minLineLength=minL, maxLineGap=gap)
        if linesP is not None:
            for l in linesP:
                ln = tuple(map(int, l[0]))
                if math.hypot(ln[2]-ln[0], ln[3]-ln[1]) >= min(H,W)//14:
                    lines.append(ln)
    lines = lines[:300]

    # vanishing point (cluster intersections)
    inters = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            ip = _line_intersection(lines[i], lines[j])
            if ip is None: continue
            if -W*0.5 <= ip[0] <= W*1.5 and -H*0.5 <= ip[1] <= H*1.5:
                inters.append(ip)
    vp = _cluster_points(inters, grid=max(20, min(H,W)//30))
    if vp is None: vp = (W/2, H/2)

    # thirds/golden deltas
    fx, fy = focal_xy
    thirds_pts = [(W/3,H/3),(2*W/3,H/3),(W/3,2*H/3),(2*W/3,2*H/3)]
    gfac = 0.618
    golden_pts = [(gfac*W,gfac*H),((1-gfac)*W,gfac*H),(gfac*W,(1-gfac)*H),((1-gfac)*W,(1-gfac)*H)]
    diag = np.hypot(W,H)
    thirds_delta = min(np.hypot(fx-x,fy-y) for (x,y) in thirds_pts)/(diag+1e-6)
    golden_delta = min(np.hypot(fx-x,fy-y) for (x,y) in golden_pts)/(diag+1e-6)

    # symmetry & edge balance
    g_f = g.astype(np.float32)/255.0
    left = g_f[:, :W//2]; right = np.fliplr(g_f[:, W - left.shape[1]:])
    mse = float(np.mean((left - right)**2))
    symmetry = float(1.0 - min(1.0, mse*4.0))
    lw, rw = float(np.sum(edges[:,:W//2])), float(np.sum(edges[:,W//2:]))
    balance = (rw - lw)/(lw + rw + 1e-6)

    return dict(
        lines=lines, vanishing_point=vp,
        thirds_delta=thirds_delta, golden_delta=golden_delta,
        symmetry=symmetry, edge_balance=balance
    )

def line_strength(lines):
    # more lines → stronger composition indicator (with soft cap)
    return norm01(len(lines), 20, 260)
# -----------------------------
# Illumination (direction + skew)
# -----------------------------
def illumination_analysis(img):
    L = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)[:,:,0].astype(np.float32)
    mu, sig = L.mean(), L.std()+1e-6
    skew = float((((L-mu)/sig)**3).mean())
    gx, gy = cv2.Sobel(L, cv2.CV_32F, 1,0), cv2.Sobel(L, cv2.CV_32F, 0,1)
    hi = (L > np.percentile(L, 90)).astype(np.uint8)
    if hi.sum() > 0:
        gx, gy = (gx*hi).sum(), (gy*hi).sum()
        ang = float((degrees(atan2(-gy, -gx)) + 360) % 360)
    else:
        ang = None
    return dict(lightness_skew=skew, light_direction_deg=ang)

def light_strength(illum):
    # if direction is None, very weak
    if illum['light_direction_deg'] is None:
        return 0.0
    # presence of direction + some skew implies visible shading
    s = abs(illum['lightness_skew'])
    return max(0.15, norm01(s, 0.1, 0.8))
# -----------------------------
# Overlays (no text on artwork)
# -----------------------------
def overlays_on_artwork(img, C, T, S, P, I):
    base = img.astype(np.float32)/255.0
    H, W = base.shape[:2]

    # strengths (0..1) for visibility/opacity
    w_tex   = texture_strength(T)
    w_de    = contrast_strength(C)
    w_sal   = saliency_strength(S['subject_mask'])
    w_lines = line_strength(P['lines'])
    w_light = light_strength(I)

    # 1) Texture (amber) — fade by w_tex
    gmap = T["gabor_map"]
    gmap = cv2.GaussianBlur(gmap, (0,0), 1.0)
    gmap = (gmap - gmap.min())/(gmap.max()-gmap.min()+1e-6)
    tex_layer = np.dstack([gmap, gmap*0.45, 0*gmap]) * (0.35 * w_tex)

    # 2) ΔE proxy (magenta) — fade by w_de
    contrast = C["contrast_map"]
    contrast = cv2.GaussianBlur(contrast, (0,0), 1.0)
    contrast = (contrast - contrast.min())/(contrast.max()-contrast.min()+1e-6)
    de_layer = np.dstack([contrast, 0*contrast, contrast]) * (0.28 * w_de)

    # 3) Saliency (soft red) — fade by w_sal
    sal = S["saliency_map"].astype(np.float32)
    if sal.shape[:2] != (H, W):
        sal = cv2.resize(sal, (W, H), interpolation=cv2.INTER_CUBIC)
    sal = cv2.normalize(sal, None, 0, 1, cv2.NORM_MINMAX)
    sal_layer = np.dstack([sal, sal*0.45, sal*0.45]) * (0.22 * w_sal)

    # 4) Lines (cyan/green/yellow) — fade by w_lines
    line_layer = np.zeros_like(base)
    for (x1,y1,x2,y2) in P["lines"][:240]:
        ang = (np.rad2deg(np.arctan2(y2-y1, x2-x1)) + 180) % 180
        col = (0,1,0) if 60<ang<120 else (0,0,1) if ang<30 or ang>150 else (1,1,0)
        cv2.line(line_layer, (x1,y1), (x2,y2), (col[2],col[1],col[0]), 1, cv2.LINE_AA)
    line_layer = cv2.GaussianBlur(line_layer, (0,0), 2) * (0.45 * w_lines)

    # blend
    comp = base*0.92 + tex_layer + de_layer + sal_layer + line_layer
    comp = np.clip(comp, 0, 1)
    comp_bgr = (comp[:,:,::-1]*255).astype(np.uint8).copy()

    # 5) Subject contour (magenta edge) — fade by w_sal
    mask = S["subject_mask"]
    cnts,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if cnts and w_sal > 0.05:
        cv2.drawContours(comp_bgr, cnts, -1, (255, 0, 255), 2)

    # 6) Focal rings — always present but thin
    fx, fy = S["focal"]
    for r in range(6, 30, 6):
        cv2.circle(comp_bgr, (int(fx), int(fy)), r, (0,0,255), 1)

    # 7) Vanishing point + light arrow — fade by w_light
    vp = P.get("vanishing_point")
    if vp:
        cv2.circle(comp_bgr, tuple(map(int, vp)), 6, (0,255,255), -1)
    if I["light_direction_deg"] is not None and w_light > 0.05:
        Hc, Wc = comp_bgr.shape[:2]
        cx, cy = Wc-60, 60; r = min(Hc,Wc)//3
        ang = I["light_direction_deg"]
        ex, ey = int(cx - r*np.cos(np.radians(ang))), int(cy + r*np.sin(np.radians(ang)))
        cv2.arrowedLine(comp_bgr, (cx,cy), (ex,ey), (0,150,255), 3, tipLength=0.1)

    return comp_bgr[:,:,::-1], dict(w_tex=w_tex, w_de=w_de, w_sal=w_sal, w_lines=w_lines, w_light=w_light)

# -----------------------------
# Fixed black analysis panel (right 400 px)
# -----------------------------
def make_hue_bar(hist, width=360, height=40):
    bar = np.zeros((height, width, 3), np.uint8)
    for x in range(width):
        h = int((x/width)*179)
        rgb = cv2.cvtColor(np.uint8([[[h, 200, 220]]]), cv2.COLOR_HSV2RGB)[0,0]
        bar[:, x] = rgb
    if hist is not None and len(hist) > 0:
        bins = len(hist)
        bin_w = max(1, width // bins)
        norm = hist.astype(np.float32)
        if norm.max() > 0: norm /= (norm.max() + 1e-6)
        for i, v in enumerate(norm):
            x1 = i*bin_w; x2 = min(width-1, (i+1)*bin_w-1)
            h_px = int(v*(height-8))
            cv2.rectangle(bar, (x1, height-1), (x2, height-1-h_px), (0,0,0), -1)
    return bar

def make_orientation_bar(angles, energies, width=360, height=40):
    bar = np.full((height, width, 3), 240, np.uint8)
    if energies is None or len(energies)==0: return bar
    e = np.array(energies, dtype=np.float32)
    e = (e - e.min())/(e.max()-e.min()+1e-6)
    for a, v in zip(angles, e):
        x = int((a/180.0)*width)
        h_px = int(v*(height-8))
        cv2.rectangle(bar, (x-2, height-1), (x+2, height-1-h_px), (60,60,60), -1)
    cv2.putText(bar, "0°", (5,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "90°", (width//2-15,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    cv2.putText(bar, "180°", (width-50,18), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    return bar

def right_panel_black(H, C, T, P, I, strengths, paragraph):
    panel = np.zeros((H, PANEL_W, 3), np.uint8)  # black background
    f = cv2.FONT_HERSHEY_SIMPLEX
    x = 16; y = 28; lh = 22

    # Title
    cv2.putText(panel, "Analysis", (x,y), f, 0.8, (255,255,255), 2); y += lh+6

    # Legend
    cv2.putText(panel, "Legend", (x,y), f, 0.7, (255,255,255), 1); y += lh
    sw = [
        ((x, y), (x+18, y+18), (0,0,255),   "Focal"),
        ((x, y+22), (x+18, y+40), (255,0,255), "Subject"),
        ((x, y+44), (x+18, y+62), (0,255,255), "Lines"),
        ((x, y+66), (x+18, y+84), (255,128,0), "Texture"),
        ((x, y+88), (x+18, y+106),(255,0,255), "ΔE"),
        ((x, y+110),(x+18, y+128),(255,255,255),"Thirds"),
        ((x, y+132),(x+18, y+150),(0,255,255), "Golden"),
        ((x, y+154),(x+18, y+172),(0,150,255), "Light→"),
        ((x, y+176),(x+18, y+194),(0,255,255), "Vanishing"),
    ]
    for (p1, p2, col, label) in sw:
        cv2.rectangle(panel, p1, p2, col, -1)
        cv2.putText(panel, label, (p2[0]+8, p2[1]-2), f, 0.55, (220,220,220), 1)
    y += 196 + 16

    # Captions (with weak/omitted notes)
    def note(strength, main, weak_text="(faint/omitted in overlay)"):
        return main if strength >= 0.08 else f"{main} {weak_text}"

    captions = [
        note(strengths['w_de'],    f"Color contrast: meanΔE {C['mean_deltaE']:.1f}, {C['harmony']} harmony."),
        note(strengths['w_tex'],   f"Texture energy: anisotropy {T['gabor_anisotropy']:.2f}, dir {T['gabor_dom_angle']:.0f}°."),
        note(strengths['w_sal'],   "Subject zone: central saliency fused."),
        note(strengths['w_lines'], f"Composition lines: {len(P['lines'])} segments; thirdsΔ {P['thirds_delta']:.3f}, goldenΔ {P['golden_delta']:.3f}."),
        note(strengths['w_light'], f"Light direction: {I['light_direction_deg']}°, skew {I['lightness_skew']:.2f}.")
    ]
    for cap in captions:
        for line in wrap_text_lines(cap, width_chars=42):
            if y+lh+10 >= H: break
            cv2.putText(panel, line, (x, y), f, 0.55, (200,200,200), 1)
            y += lh
        y += 6
        if y >= H-120: break

    # Bars (hue + orientation) if space
    hb = make_hue_bar(C['hue_hist'], width=PANEL_W-32, height=40)
    ob = make_orientation_bar(T['gabor_angles'], T['gabor_energy'], width=PANEL_W-32, height=40)
    if y+40*2+16 < H:
        cv2.putText(panel, "Hue distribution", (x, y), f, 0.55, (255,255,255), 1); y += lh
        panel[y:y+hb.shape[0], x:x+hb.shape[1]] = hb; y += hb.shape[0] + 6
        cv2.putText(panel, "Orientation energy", (x, y), f, 0.55, (255,255,255), 1); y += lh
        panel[y:y+ob.shape[0], x:x+ob.shape[1]] = ob; y += ob.shape[0] + 6

    # Curatorial summary at bottom if space remains, else squeeze above
    para_lines = wrap_text_lines(paragraph, width_chars=44)
    y_para = min(y+8, H- (len(para_lines)+1)*lh - 8)
    cv2.putText(panel, "Curatorial Summary", (x, y_para), f, 0.62, (255,255,255), 1)
    y_para += lh
    for pl in para_lines:
        if y_para >= H-6: break
        cv2.putText(panel, pl, (x, y_para), f, 0.55, (230,230,230), 1)
        y_para += lh

    return panel
# -----------------------------
# Curatorial summary (paragraph, encouraging)
# -----------------------------
def curatorial_paragraph(M):
    C, T, P, I = M["color"], M["texture"], M["composition"], M["illumination"]
    # color
    tone = {"analogous":"a gentle, closely-related set of hues",
            "complementary":"a lively conversation of contrasting colors"}.get(C["harmony"], "a balanced, variegated palette")
    if C["mean_deltaE"] > 25: contrast = "confident, high-contrast shifts"
    elif C["mean_deltaE"] > 12: contrast = "measured points of contrast"
    else: contrast = "soft transitions that invite lingering attention"
    # texture
    texture_phrase = (f"Texture carries a directional current near {T['gabor_dom_angle']:.0f}°, guiding the eye across the surface"
                      if T["gabor_anisotropy"] > 0.5 else
                      "Texture remains even and poised, letting color and structure lead")
    # composition
    if P["golden_delta"] < 0.12:
        comp_phrase = "The focal area brushes against classical golden proportion."
    elif P["thirds_delta"] < 0.12:
        comp_phrase = "The focal area settles near a rule-of-thirds point, feeling intuitive and open."
    else:
        comp_phrase = "The focal pull resists fixed proportion, encouraging open exploration."
    sym_phrase = ("A notable symmetry steadies the frame."
                  if P["symmetry"] > 0.8 else
                  "Partial symmetry tempers the motion."
                  if P["symmetry"] > 0.5 else
                  "Asymmetry keeps the image lively and immediate.")
    balance_phrase = ("Edge-weight remains steady."
                      if abs(P["edge_balance"]) < 0.1 else
                      "A deliberate imbalance animates the frame.")
    # illumination
    light_phrase = (f"Light arrives from roughly {int(I['light_direction_deg'])}°, gently shaping form."
                    if I["light_direction_deg"] is not None else
                    "Light diffuses evenly, allowing the whole surface to breathe.")
    mood = "bright and open" if I["lightness_skew"] > 0.5 else \
           "quietly contemplative" if I["lightness_skew"] < -0.5 else \
           "balanced and steady"
    paragraph = (
        f"This work encourages an attentive, unhurried look. The palette offers {tone}, with {contrast}. "
        f"{texture_phrase}. {comp_phrase} {sym_phrase} {balance_phrase} "
        f"{light_phrase} Overall the mood feels {mood}, inviting the eye to move, return, and discover nuance."
    )
    return paragraph

# -----------------------------
# Main pipeline
# -----------------------------
def analyze_artwork_v16(path, verbose=True, thorough=True):
    t0 = time.time()
    if verbose:
        print(f"🖼️ Loading image: {path}")
    img = load_rgb(path)
    H0, W0 = img.shape[:2]
    if verbose:
        print(f"   • Image size: {W0}×{H0}")

    if thorough:
        img, _ = resize_min_pixels(img, target_pixels=1_600_000)
        H, W = img.shape[:2]
        if verbose and (H,W)!=(H0,W0):
            print(f"   • Rescaled for fidelity: {W}×{H}")
    else:
        H, W = H0, W0

    # Analyses
    if verbose: print("🎨 Color…", end="", flush=True)
    C = color_analysis(img);  print(" ✓")
    if verbose: print("🪶 Texture…", end="", flush=True)
    T = gabor_pyramid(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), scales=(3,5,7,9,11,13), orientations=12);  print(" ✓")
    if verbose: print("🔥 Saliency…", end="", flush=True)
    S = saliency_analysis(img);  print(" ✓")
    if verbose: print("📐 Composition…", end="", flush=True)
    P = composition_analysis(img, S['focal']);  print(" ✓")
    if verbose: print("💡 Illumination…", end="", flush=True)
    I = illumination_analysis(img);  print(" ✓")

    # Overlays & strengths
    art_over, strengths = overlays_on_artwork(img, C, T, S, P, I)
    paragraph = curatorial_paragraph(dict(color=C, texture=T, composition=P, illumination=I))
    panel = right_panel_black(art_over.shape[0], C, T, P, I, strengths, paragraph)

    # Stack horizontally (art | panel)
    final = np.hstack([art_over, panel])
    final = safe_gamma(final)

    # Output
    out_dir = os.path.splitext(path)[0] + "_analysis"
    ensure_dir(out_dir)
    out_png = os.path.join(out_dir, "overlay_final_composite.png")
    cv2.imwrite(out_png, final)

    # Save summary & metrics
    out_txt = os.path.join(out_dir, "curatorial_summary.txt")
    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(paragraph + "\n")

    out_json = os.path.join(out_dir, "metrics.json")
    with open(out_json, "w", encoding="utf-8") as f:
        json.dump(json_safe(dict(color=C, texture=T, saliency=S, composition=P, illumination=I, strengths=strengths)), f, indent=2)

    if verbose:
        print(f"\n✅ Analysis complete in {time.time()-t0:.2f}s")
        print(f"📁 Saved to: {out_dir}")

# -----------------------------
# Run
# -----------------------------
if __name__ == "__main__":
    # ⬇️ Set your image path here:
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/VG_-_skeleton.jpg"
    analyze_artwork_v16(IMAGE_PATH, verbose=True, thorough=True)


🖼️ Loading image: /Users/alievanayasso/Documents/SlowMA/VG_-_skeleton.jpg
   • Image size: 2481×3289
🎨 Color… ✓
🪶 Texture… ✓
🔥 Saliency… ✓
📐 Composition… ✓
💡 Illumination… ✓

✅ Analysis complete in 59.69s
📁 Saved to: /Users/alievanayasso/Documents/SlowMA/VG_-_skeleton_analysis
